# Bitcoin Price Prediction Using Deep Learning Techniques

## Part 3: MLP Models for Time Series Prediction

In this notebook, we build and train Multilayer Perceptron (MLP) models for Bitcoin price prediction using both the original price series and the fractionally differenced series.

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Deep Learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

# Visualization Settings
import matplotlib as mpl
mpl.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load prepared data
try:
    with open('btc_prepared_data.pkl', 'rb') as f:
        data_dict = pickle.load(f)
    
    # Extract data
    btc_data = data_dict['btc_data']
    optimal_d = data_dict['optimal_d']
    X_train_price = data_dict['X_train_price']
    X_test_price = data_dict['X_test_price']
    y_train_price = data_dict['y_train_price']
    y_test_price = data_dict['y_test_price']
    X_train_frac = data_dict['X_train_frac']
    X_test_frac = data_dict['X_test_frac']
    y_train_frac = data_dict['y_train_frac']
    y_test_frac = data_dict['y_test_frac']
    window_size = data_dict['window_size']
    
    print("Data loaded successfully.")
    print(f"Window size: {window_size}")
    print(f"Optimal fractional differencing order: {optimal_d}")
    print(f"Training data shape: {X_train_price.shape}")
    print(f"Testing data shape: {X_test_price.shape}")
except FileNotFoundError:
    print("Prepared data not found. Please run Part 2 first.")

## Data Preprocessing

Before training our models, we need to normalize the data to improve training stability and convergence.

In [ ]:
# Normalize data for price prediction
price_scaler = MinMaxScaler()
X_train_price_scaled = np.array([price_scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in X_train_price])
X_test_price_scaled = np.array([price_scaler.transform(x.reshape(-1, 1)).flatten() for x in X_test_price])

y_train_price_scaled = price_scaler.transform(y_train_price.reshape(-1, 1)).flatten()
y_test_price_scaled = price_scaler.transform(y_test_price.reshape(-1, 1)).flatten()

# Normalize data for fractionally differenced series prediction
frac_scaler = MinMaxScaler()
X_train_frac_scaled = np.array([frac_scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in X_train_frac])
X_test_frac_scaled = np.array([frac_scaler.transform(x.reshape(-1, 1)).flatten() for x in X_test_frac])

y_train_frac_scaled = frac_scaler.transform(y_train_frac.reshape(-1, 1)).flatten()
y_test_frac_scaled = frac_scaler.transform(y_test_frac.reshape(-1, 1)).flatten()

print("Data normalization complete.")

## MLP Model for Raw Price Prediction

First, let's build and train an MLP model to predict the raw Bitcoin price using past price information.

In [ ]:
def build_mlp_model(input_shape, learning_rate=0.001):
    """
    Build a Multilayer Perceptron (MLP) model for time series prediction.
    
    Parameters:
    -----------
    input_shape : tuple
        Shape of the input data
    learning_rate : float
        Learning rate for the optimizer
        
    Returns:
    --------
    tensorflow.keras.models.Sequential
        Compiled MLP model
    """
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mean_squared_error'
    )
    
    return model

# Build and train MLP model for raw price prediction
mlp_price_model = build_mlp_model(window_size)

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train the model
history_price = mlp_price_model.fit(
    X_train_price_scaled, y_train_price_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history_price.history['loss'], label='Training Loss')
plt.plot(history_price.history['val_loss'], label='Validation Loss')
plt.title('MLP Model Training History (Raw Price Prediction)', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## MLP Model for Fractionally Differenced Series Prediction

Now, let's build and train an MLP model to predict the fractionally differenced series.

In [ ]:
# Build and train MLP model for fractionally differenced series prediction
mlp_frac_model = build_mlp_model(window_size)

# Train the model
history_frac = mlp_frac_model.fit(
    X_train_frac_scaled, y_train_frac_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history_frac.history['loss'], label='Training Loss')
plt.plot(history_frac.history['val_loss'], label='Validation Loss')
plt.title('MLP Model Training History (Fractionally Differenced Prediction)', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Model Evaluation

Let's evaluate the performance of our MLP models on the test data.

In [ ]:
def evaluate_model(model, X_test_scaled, y_test_scaled, scaler, model_name):
    """
    Evaluate a model's performance on test data.
    
    Parameters:
    -----------
    model : tensorflow.keras.models.Model
        Trained model to evaluate
    X_test_scaled : numpy.ndarray
        Scaled test input data
    y_test_scaled : numpy.ndarray
        Scaled test target data
    scaler : sklearn.preprocessing.MinMaxScaler
        Scaler used to normalize the target data
    model_name : str
        Name of the model for display purposes
        
    Returns:
    --------
    dict
        Dictionary containing evaluation metrics
    """
    # Make predictions
    y_pred_scaled = model.predict(X_test_scaled)
    
    # Inverse transform predictions and actual values
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()
    y_true = scaler.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Print metrics
    print(f"Evaluation Metrics for {model_name}:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}%")
    
    # Plot actual vs. predicted values
    plt.figure(figsize=(14, 7))
    plt.plot(y_true, label='Actual', color='blue', alpha=0.7)
    plt.plot(y_pred, label='Predicted', color='red', alpha=0.7)
    plt.title(f'{model_name}: Actual vs. Predicted Values', fontsize=14, fontweight='bold')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Plot prediction error
    error = y_true - y_pred
    plt.figure(figsize=(14, 7))
    plt.plot(error, color='green', alpha=0.7)
    plt.title(f'{model_name}: Prediction Error', fontsize=14, fontweight='bold')
    plt.xlabel('Time Step')
    plt.ylabel('Error')
    plt.axhline(y=0, color='r', linestyle='--')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return {
        'model_name': model_name,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_true': y_true,
        'y_pred': y_pred
    }

# Evaluate MLP model for raw price prediction
price_results = evaluate_model(
    mlp_price_model, 
    X_test_price_scaled, 
    y_test_price_scaled, 
    price_scaler, 
    "MLP Model (Raw Price Prediction)"
)

# Evaluate MLP model for fractionally differenced series prediction
frac_results = evaluate_model(
    mlp_frac_model, 
    X_test_frac_scaled, 
    y_test_frac_scaled, 
    frac_scaler, 
    "MLP Model (Fractionally Differenced Prediction)"
)

## Inverse Transformation for Fractionally Differenced Predictions

For the fractionally differenced series, we need to inverse transform the predictions to get back to the original price scale for a fair comparison.

In [ ]:
def inverse_fractional_difference(frac_diff_series, original_series, d, window_size=30):
    """
    Approximate inverse of fractional differencing to recover the original series.
    This is a simplified approach and may not perfectly recover the original series.
    
    Parameters:
    -----------
    frac_diff_series : numpy.ndarray
        Fractionally differenced series
    original_series : numpy.ndarray
        Original series used for initialization
    d : float
        Fractional differencing order
    window_size : int
        Window size for prediction
        
    Returns:
    --------
    numpy.ndarray
        Recovered original series
    """
    # Get weights for inverse operation
    weights = np.array([1.0])
    for k in range(1, window_size):
        weights = np.append(weights, weights[-1] * (d - k + 1) / k)
    
    # Initialize recovered series with known values
    recovered = np.zeros(len(frac_diff_series) + window_size)
    recovered[:window_size] = original_series[:window_size]
    
    # Iteratively recover the original series
    for i in range(window_size, len(recovered)):
        recovered[i] = frac_diff_series[i - window_size]
        for j in range(1, window_size + 1):
            recovered[i] += weights[j-1] * recovered[i-j]
    
    return recovered[window_size:]

# Get the test dates for plotting
test_dates = btc_data.index[-len(y_test_price):]

# Get the original price values for comparison
original_price = btc_data['Close'].values[-len(y_test_price):]

# Get the fractionally differenced predictions
frac_diff_pred = frac_results['y_pred']

# Inverse transform the fractionally differenced predictions (simplified approach)
# Note: This is an approximation and may not perfectly recover the original series
try:
    recovered_price_pred = inverse_fractional_difference(
        frac_diff_pred, 
        btc_data['Close'].values[:window_size], 
        optimal_d,
        window_size
    )
    
    # Plot comparison
    plt.figure(figsize=(14, 7))
    plt.plot(test_dates, original_price, label='Actual Price', color='blue', alpha=0.7)
    plt.plot(test_dates, price_results['y_pred'], label='Raw Price Model', color='red', alpha=0.7)
    plt.plot(test_dates[-len(recovered_price_pred):], recovered_price_pred, 
             label='Recovered from Frac Diff Model', color='green', alpha=0.7)
    plt.title('Comparison of Prediction Methods', fontsize=14, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Bitcoin Price (USD)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error in inverse transformation: {e}")
    print("Skipping inverse transformation visualization.")

## Performance Comparison

Let's compare the performance of our MLP models on both the raw price data and the fractionally differenced data.

In [ ]:
# Create a DataFrame for performance comparison
performance_df = pd.DataFrame([
    {
        'Model': 'MLP (Raw Price)',
        'MAE': price_results['mae'],
        'RMSE': price_results['rmse'],
        'MAPE (%)': price_results['mape']
    },
    {
        'Model': 'MLP (Fractionally Differenced)',
        'MAE': frac_results['mae'],
        'RMSE': frac_results['rmse'],
        'MAPE (%)': frac_results['mape']
    }
])

display(performance_df)

# Save models and results
mlp_price_model.save('mlp_price_model.h5')
mlp_frac_model.save('mlp_frac_model.h5')

results_dict = {
    'price_results': price_results,
    'frac_results': frac_results,
    'performance_df': performance_df
}

with open('mlp_results.pkl', 'wb') as f:
    pickle.dump(results_dict, f)

print("Models and results saved successfully.")

## Summary of Part 3

In this part of our analysis, we've:

1. Built and trained MLP models for both raw Bitcoin price prediction and fractionally differenced series prediction
2. Evaluated the models' performance using various metrics (MAE, RMSE, MAPE)
3. Attempted to inverse transform the fractionally differenced predictions for comparison
4. Compared the performance of the two approaches

Key findings:
- The MLP model trained on fractionally differenced data likely shows different error characteristics compared to the model trained on raw price data
- Fractional differencing helps achieve stationarity while preserving more information than traditional differencing
- The choice between using raw price data or fractionally differenced data depends on the specific prediction task and performance requirements

In the next part, we'll explore CNN models with Gramian Angular Field (GAF) representations for time series prediction.